### Testing APIs

Working out the twitter and llm apis.

Currently using the free version of Twitter/X api and Google Flash 2.0 free API. 

In [93]:
import requests, os
from secrets import google_key, twitter_token

twitter_header =  {"Authorization": f"Bearer {twitter_token}"}
google_header = {"Content-Type": "application/json"}
query = "Brazil landslide"

### Twitter API Testing

Limitations as of right now with the free version: I can only look at recent tweets + there's a really long cooldown time between requests. I would estimate like 30 minutes unless my sense of time is scuffed. 

In [85]:
def search_recent(query, max_results= 15):
    url  = "https://api.twitter.com/2/tweets/search/recent"
    params = {
        "query": query,
        "max_results": max_results,
        "tweet.fields": "created_at,text,author_id"
    }
    response = requests.get(url, headers=twitter_header, params= params)
    return response.json()

In [90]:
tweets = search_recent(query)

In [91]:
print(tweets)

{'data': [{'edit_history_tweet_ids': ['1967309237052395872'], 'created_at': '2025-09-14T19:27:27.000Z', 'author_id': '1209135215395983362', 'id': '1967309237052395872', 'text': 'RT @lcat0919: @GeneralMCNews This is sick. Bolsonaro won the election in a landslide. Just sick. Rise up Brazil! Take your country back.'}, {'edit_history_tweet_ids': ['1967212878081179981'], 'created_at': '2025-09-14T13:04:33.000Z', 'author_id': '1548935043481014272', 'id': '1967212878081179981', 'text': 'RT @irene_makarenko: #ClimateAwareness #ClimateCrisis #Weather\nSeptember 12_4⃣\n\n🌪️Water tornado near Fort Myers Beach, #Florida, USA 🇺🇸\n🌪️🔥…'}, {'edit_history_tweet_ids': ['1967095064993435800'], 'created_at': '2025-09-14T05:16:25.000Z', 'author_id': '1638842378353033216', 'id': '1967095064993435800', 'text': '#ClimateAwareness #ClimateCrisis #Weather\nSeptember 12_4⃣\n\n🌪️Water tornado near Fort Myers Beach, #Florida, USA 🇺🇸\n🌪️🔥Fire whirlwind broke out at a fire in São Paulo, #Brazil 🇧🇷\n🌩️Thunderstorms

This code was to transfer my results into a json so I can save it, but lowkey I already wrote over the variable before I could save what I wanted so I just ended up copying and pasting into a document. 

In [92]:
import json 
with open("tweets01.json", "w", encoding = "utf-8") as f: 
    json.dump(tweets, f, ensure_ascii= False, indent = 2)

### LLM Testing

In this section, I will test the LLM using a free google api key. I will also use a fake tweet as an argument for the llm prompt before I have 

In [55]:
fake_tweet = """SERIOUS: Avenida Brasil, at the height of Passarela 25, in Irajá, completely flooded.

This is just the most important expressway in the city of Rio de Janeiro."""


With the real tweets, I definitely need to do some preprocessing haha, I can probably use a preprocess function from one of the research papers though, or just make my own. 

In [71]:
def prompt_gemini(text):
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={google_key}"
    data = {
        "contents": [
            {
                "parts": [
                    {"text": text}
                ]
            }
        ]
    }
    response = requests.post(url, headers = google_header, json= data)

    if response.status_code == 200:
        result = response.json()
        try:
            return result["candidates"][0]["content"]["parts"][0]["text"]
        except (KeyError, IndexError):
            print("Unexpected response format:", result)
            return None
    else:
        print("Error:", response.status_code, response.text)
        return None

In [67]:
disaster_types = [
        "Earthquake",
    "Tsunami",
    "Flood",
    "Hurricane",
    "Wildfire",
    "Drought",
    "Heatwave",
    "Landslide",
    "Volcano",
    "Tornado",
    "Pandemic",
    "Famine",
    "Conflict",
    "Cyberattack",
    "Blackout",
    "Chemical Spill",
    "Nuclear Accident",
    "Industrial Accident",
    "Mass Shooting",
    "Explosion",
    "Other",
    "N/A"
]

severity_levels = [
    "Severe damage",
"Mild damage",

"Little or no damage",

"Do not know or cannot judge"
]

informative_levels = [
    "True", 
    "False"
]

impact = [
    "Affected individuals",
    "Infrastructure and utility damage",
    "Injured or dead people",
    "Missing or found people",
    "Rescue, volunteering, or donation effort",
    "Vehicle damage",
    "Other relevant information",
    "Not relevant"
]



In [72]:
answer = prompt_gemini(
   f"""You are a helpful assistant that classifies disaster-related tweets. 
Classify the following tweet into the categories of main disaster type, severity, informative, and impact. 
Use the provided keywords for each category to determine the appropriate label. 
Sentiment must be either positive, negative, or neutral. Location mentioned should be a country or city name if mentioned in the tweet. 
Respond with the appropriate JSON format. 

Disaster Types Values List: {', '.join(disaster_types)} 
Severity Levels Values List: {', '.join(severity_levels)} 
Informative Levels Values List: {', '.join(informative_levels)} 
Impact Values List: {', '.join(impact)} 

Tweet: {fake_tweet}
"""
)

In [73]:
print(answer)

```json
{
  "Disaster Type": "Flood",
  "Severity Level": "Severe damage",
  "Informative": "True",
  "Impact": "Infrastructure and utility damage",
  "Sentiment": "Negative",
  "Location Mentioned": "Rio de Janeiro"
}
```


### Conclusion

Both apis seem to work fine haha, definitely limiting right now since I can't make too many requests.